In [32]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [45]:
options = webdriver.ChromeOptions()
options.add_argument("disable-gpu") #그래픽 성능 낮춰서 크롤링 성능 쪼금 높이기
options.add_argument("lang=ko_KR") # 사이트 주언어

driver = webdriver.Chrome(chrome_options=options)
driver.set_window_size(800, 600)

C:\Users\admin\AppData\Local\Temp\ipykernel_16000\3262268338.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [46]:
Url = 'https://www.youtube.com/watch?v=PweirbWOHM8'
driver.get(Url)
time.sleep(3)

일본 첫날! 오사카 첫인상 방문
https://thecitizen.kr/bbs/board.php?bo_table=free&wr_id=216

In [47]:
# 크롤링 목표 : 해당 영상에 대한 댓글 id, 댓글 내용, 댓글의 좋아요 개수, 날짜 추출
data_list = []
# 스크롤 내리기
body = driver.find_element(By.TAG_NAME, 'body')
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_page_height == last_page_height:
        break
    last_page_height = new_page_height

try:
    driver.find_element(By.CSS_SELECTOR,"#dismiss-button > a").click()
except:
    pass
buttons = driver.find_elements(By.CSS_SELECTOR, "#more-replies > a")

time.sleep(1.5)

for button in buttons:
    driver.execute_script("arguments[0].click()", button)
    button.send_keys(Keys.ENTER)
    time.sleep(2)
    button.click()

   
# bs4 html 파싱
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id)

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment)
    
    

pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('youtube.xlsx')

# 엑셀 불러와서, 영어 댓글만 거르기